In [44]:
import pandas as pd
import xarray as xr
from pandas import to_datetime as tdt
import numpy as np

names_of_conc=['no2_conc','o3_conc','so2_conc','pm2p5_conc','pm10_conc','co_conc']
names_of_infiles=['no2_spring.nc','o3_spring.nc','so2_spring.nc','pm2.5_spring.nc','pm10_spring.nc','co_spring.nc']
names_of_outfiles=['no2_for_matrix.txt','o3_for_matrix.txt','so2_for_matrix.txt','pm2p5_for_matrix.txt','pm10_for_matrix.txt','co_for_matrix.txt']

def do_everything(days,infile,name_chemical,outfile):
    days=days*6
    ds = xr.open_dataset(infile)
    df_ = ds.to_dataframe().reset_index()
    start_date = tdt('2021-03-01')
    df_['time'] = start_date + df_['time']
    df = df_[(df_['latitude'] >= 50.3)  & (df_['latitude'] <= 51.7) & (df_['longitude'] <= 29.8) & (df_['longitude'] >= 22)]
    time=np.array(df['time'])[:days]
    vykydy=[]
    for i in range(days):
        dd=df[df['time']==time[i]]
        chemical=np.array(dd[name_chemical])
        vykydy.append((chemical.sum()/1008))
    result_data = []
    for i in range(0,days,6):
        ser = (vykydy[i] + vykydy[i+1] + vykydy[i+2] +vykydy[i+3] + vykydy[i+4] + vykydy[i+5])/6
        result_data.append(ser)
    len_of_result_data=len(result_data)
    np.savetxt(outfile, result_data, delimiter='\n', fmt='%1.2f')
for i in range (5):
    do_everything(92,names_of_infiles[i],names_of_conc[i],names_of_outfiles[i])


In [42]:
import pandas as pd
import xarray as xr
from pandas import to_datetime as tdt
import numpy as np


names_of_infiles=['no2_spring.nc','o3_spring.nc','so2_spring.nc','pm2.5_spring.nc','pm10_spring.nc','co_spring.nc']


def prep(file_paths: list, lat_min=None, lat_max=None, long_min=None, long_max=None) -> pd.DataFrame:
    first_df = xr.open_dataset(file_paths[0])
    first_df = first_df.to_dataframe().reset_index().reset_index()
    start_date = tdt('2021-03-01')
    first_df['time'] = start_date + first_df['time']

    first_df = first_df[(first_df['latitude'] >= lat_min) & (first_df['latitude'] <= lat_max)
              & (first_df['longitude'] >= long_min)& (first_df['longitude'] <= long_max)]
    first_df = first_df.drop(columns=['longitude','latitude','level','time'])
    res = first_df.copy(deep=True)

    for f in file_paths[1:]:
        _df = xr.open_dataset(f)
        _df = _df.to_dataframe().reset_index().reset_index()
        _df['time'] = start_date + _df['time']

        _df = _df[(_df['latitude'] >= lat_min) & (_df['latitude'] <= lat_max)
                   & (_df['longitude'] >= long_min)& (_df['longitude'] <= long_max)]
        _df = _df.drop(columns=['longitude','latitude','level','time'])
        res = res.merge(_df, on='index', how='left')

    return res.drop(columns='index')


In [43]:
prep(names_of_infiles, lat_min=51.4, lat_max=51.7, long_min=22, long_max=22.9)

,no2_conc,o3_conc,so2_conc,pm2p5_conc,pm10_conc,co_conc
0,4.786252,52.067883,2.109083,8.933200,12.472599,201.739655
1,5.183743,48.610947,2.101096,8.116792,9.810987,201.875107
2,5.423501,53.156441,3.716337,7.138286,11.403473,183.057281
3,5.637799,53.092793,5.756590,9.794201,11.825789,183.955597
4,13.023629,43.032497,6.437304,14.760189,19.231266,234.732834
...,...,...,...,...,...,...
4963,1.772491,38.981445,0.932460,5.158408,7.152264,143.861252
4964,0.784834,65.498566,1.294439,4.550786,9.324713,135.610825
4965,0.600636,71.967300,0.717476,4.736873,5.940708,134.872620
4966,1.190074,75.319412,1.142497,4.406613,5.371469,132.936523


In [ ]:
df_no2 = xr.open_dataset('no2_spring.nc')
df_o3 = xr.open_dataset('o3_spring.nc')

df_ = pd.to_dataframe().reset_index().reset_index()
start_date = tdt('2021-03-01')
df_['time'] = start_date + df_['time']
dd=df_[ (df_['latitude'] >=51.4) & (df_['latitude'] <=51.7) & (df_['longitude'] >=22)& (df_['longitude'] <=22.9)]
dd=dd.drop(columns=['longitude','latitude','level','time'])
dd

In [ ]:
prep(['test.nc', 'test2.nc'], lat_min=51.4, lat_max=51.7, long_min=22, long_max=22.9).to_csv('res.csv')

In [32]:
df_no2 = xr.open_dataset('no2_spring.nc')
df_o3 = xr.open_dataset('o3_spring.nc')

df_ = ds.to_dataframe().reset_index().reset_index()
start_date = tdt('2021-03-01')
df_['time'] = start_date + df_['time']
dd=df_[ (df_['latitude'] >=51.4) & (df_['latitude'] <=51.7) & (df_['longitude'] >=22)& (df_['longitude'] <=22.9)]
dd=dd.drop(columns=['longitude','latitude','level','time'])
dd

,index,no2_conc
0,0,4.786252
1,1,5.183743
2,2,5.423501
3,3,5.637799
4,4,13.023629
...,...,...
63475,63475,1.772491
63476,63476,0.784834
63477,63477,0.600636
63478,63478,1.190074


In [26]:
dd

,longitude,latitude,level,time,no2_conc
0,22.65,51.650002,0.0,2021-03-01 01:00:00,4.786252
1,22.65,51.650002,0.0,2021-03-01 05:00:00,5.183743
2,22.65,51.650002,0.0,2021-03-01 09:00:00,5.423501
3,22.65,51.650002,0.0,2021-03-01 13:00:00,5.637799
4,22.65,51.650002,0.0,2021-03-01 17:00:00,13.023629
...,...,...,...,...,...
63475,22.85,51.450001,0.0,2021-05-31 05:00:00,1.772491
63476,22.85,51.450001,0.0,2021-05-31 09:00:00,0.784834
63477,22.85,51.450001,0.0,2021-05-31 13:00:00,0.600636
63478,22.85,51.450001,0.0,2021-05-31 17:00:00,1.190074


In [31]:
ds = xr.open_dataset('o3_spring.nc')
df_ = ds.to_dataframe().reset_index().reset_index()
start_date = tdt('2021-03-01')
df_['time'] = start_date + df_['time']
dd1=df_[ (df_['latitude'] >=51.4) & (df_['latitude'] <=51.7) & (df_['longitude'] >=22)& (df_['longitude'] <=22.9)]
dd1=dd1.drop(columns=['longitude','latitude','level','time'])
dd1



,index,o3_conc
0,0,52.067883
1,1,48.610947
2,2,53.156441
3,3,53.092793
4,4,43.032497
...,...,...
63475,63475,38.981445
63476,63476,65.498566
63477,63477,71.967300
63478,63478,75.319412


In [ ]:
dd1.merge(dd, on='index', how='left')
